In [3]:
from pathlib import Path

import wandb
import torch
from lightning.pytorch import loggers, utilities, Trainer

from model import Digits
from data_module import MNISTDataModule

In [4]:
root_path = Path('../')
dm = MNISTDataModule(data_dir=(root_path / 'data').as_posix())

In [5]:
log_dir = root_path / 'logs'
logger = loggers.WandbLogger(
    project='digits',
    name='evaluation',
    save_dir=log_dir,
    log_model='all',
)

artifact_dir = root_path / 'artifacts'
artifact_dir.mkdir(exist_ok=True)
model_path = Path(loggers.WandbLogger.download_artifact(
    artifact='sampath017/digits/model-74c3j2pz:v12',
    artifact_type='model',
    save_dir=artifact_dir
)) # type: ignore

model_path = model_path / 'model.ckpt'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sampath017. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


In [6]:
model = Digits.load_from_checkpoint(model_path, map_location=torch.device('cpu'))
utilities.model_summary.ModelSummary(model)

c:\Dev\digits\.venv\Lib\site-packages\lightning\pytorch\utilities\migration\utils.py:55: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.8, which is newer than your current Lightning version: v2.0.7
  rank_zero_warn(


  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 322 K 
-------------------------------------
322 K     Trainable params
0         Non-trainable params
322 K     Total params
1.288     Total estimated model params size (MB)

In [7]:
trainer = Trainer(
    inference_mode=True,
    logger=logger,
    enable_model_summary=False
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
trainer.test(model, datamodule=dm);

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     99.31999969482422     │
│         test_loss         │    0.02460070513188839    │
└───────────────────────────┴───────────────────────────┘

In [9]:
wandb.finish()

epoch,▁
test_acc,▁
test_loss,▁
trainer/global_step,▁
epoch,0
test_acc,99.32
test_loss,0.0246
trainer/global_step,0
